# 학업탄력성 영향요인 연구
@author: sjh

## 1. 패키지 및 데이터 로드

In [7]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
from tqdm import tqdm
from sys import platform

import warnings
warnings.filterwarnings('ignore')

# unicode minus를 사용하지 않기 위한 설정 (minus 깨짐현상 방지)
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams["figure.autolayout"] = True

# 설치된 폰트 출력
import matplotlib.font_manager as fm
font_list = [font.name for font in fm.fontManager.ttflist]
if 'darwin' in platform:
    plt.rcParams['font.family'] = 'Malgun Gothic'
else:
    plt.rcParams['font.family'] = 'Malgun Gothic'

# mixed effect randomforest
# !pip install pyreadstat
import merf

BASE_DIR = os.getcwd()
print('>> Current OS: ', platform)
print('>> Current WD: ', BASE_DIR)

>> Current OS:  win32
>> Current WD:  c:\Users\jhun1\Proj\Research\MixedRF


In [9]:
class Load:
    def __init__(self, filePath):
        self.rawDf = pd.read_spss(os.path.join(filePath))

data = Load(r"C:\Users\jhun1\Dropbox\[3]Project\[혼합효과 랜덤포레스트_2022]\PISA2018\STU\CY07_MSU_STU_QQQ.sav")

KeyboardInterrupt: 

In [ ]:
data

## 2. 전처리

In [ ]:
class PreProcessing:
    def __init__(self):
        pass

    # 분석에 사용할 변수선정
    def featureSelection(self):
        pass

    # 학생별 학업탄력성 점수 계산
    def dummyCoding(self):
        pass

    # 결측치 대체방법 
    # 1. 00% 이상 결측은 아예 제외(listwise)
    # 2-1. rubin multiple imputation
    # 2-2. breiman na.roughfix 
    def naHandler(self):
        pass

    # 더미코딩
    # 2. 비서열 범주변수: oneHot encoding
    def dummyCoding(self):
        pass

    # 기술통계
    def descriptive(self):
        pass
    

## 3. 분석
### 3.1 Merf
#### 3.1.1 training

#### 3.1.2 evaluation

## 3.2 suplementary anlaysis